In [1]:
import numpy as np  # noqa
import pandas as pd  # noqa
from pandas import DataFrame

# --------------------------------
# Add your lib to import here
import talib.abstract as ta
from datetime import datetime, timedelta, timezone
import ccxt
import math
from enum import Enum
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [2]:
ex = ccxt.binance()
timeframe = "5m"
limit = 1000

def ohlcv_info(ex:ccxt.Exchange,timeframe,limit):
    x = ex.fetch_ohlcv('1000BONK/USDT:USDT', timeframe, limit=limit)
    df = pd.DataFrame(x, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['date'], unit='ms')
    # print(f'*** {datetime.now()}  ***\n {df.tail(10)}\n')
    return df
dataframe = ohlcv_info(ex,timeframe,limit)
dataframe = dataframe[:len(dataframe)-1]

In [3]:
def normalize_origin(value, min_val, max_val,df):
    index = value.name
    src = pd.Series(df[:index+1])

    historic_min = 10e10
    historic_max = -10e10

    src_filled_min = src.fillna(historic_min)
    src_filled_max = src.fillna(historic_max)
    historic_min = min(src_filled_min.min(), historic_min) if not pd.isna(src_filled_min.min()) else historic_min
    historic_max = max(src_filled_max.max(), historic_max) if not pd.isna(src_filled_max.max()) else historic_max

    normalized_src = (min_val + (max_val - min_val) * (src[index] - historic_min)) / max((historic_max - historic_min), 10e-10)
    return normalized_src


def normalize_optimized(min_val, max_val,source:pd.Series):
    source = source.to_numpy()
    historic_min = 10e10
    historic_max = -10e10

    src_filled_min = np.nan_to_num(source,historic_min) #source.fillna(historic_min)
    src_filled_max = np.nan_to_num(source,historic_max) #source.fillna(historic_max)
    historic_min = np.minimum.accumulate(src_filled_min)
    historic_max = np.maximum.accumulate(src_filled_max)

    division_value = np.maximum((historic_max - historic_min),10e-10)
    normalized_src = (min_val + (max_val - min_val) * (source - historic_min)) / division_value
    return normalized_src

startTime = datetime.now()
# origin_data = dataframe.apply((lambda x : normalize_origin(x,0,1,dataframe['high'])),axis=1)
improved_data = pd.Series(normalize_optimized(0,1,dataframe['high']))
# test_data:pd.Series = origin_data == improved_data
endTime = datetime.now()
# print(test_data.value_counts())
print(f"speed is {round((endTime-startTime).microseconds * 0.001, 4)}ms")

speed is 2.611ms


In [4]:
def calculate_cci_origin(dataframe,index,diff,ma,p):
    close_array = dataframe['close'].to_numpy()
    close_array = close_array[:index.name]
    diff = diff.to_numpy()
    diff = diff[index.name]
    ma = ma.to_numpy()
    ma = ma[index.name]

    if(len(close_array) < p-1):
        return np.nan

    s = 0

    for i in range(len(close_array),len(close_array)-p,-1):
        s = s + abs(dataframe['close'][i] - ma)

    mad = s / p

    mcci = diff/mad/0.015
    
    return mcci

def calculate_cci_improved(source1:pd.Series,source2:pd.Series,source3:pd.Series,length) -> pd.Series:
    source1:np.ndarray = source1.to_numpy()
    source2:np.ndarray = source2.to_numpy()
    source3:np.ndarray = source3.to_numpy()
    
    windows = np.lib.stride_tricks.sliding_window_view(source1, window_shape=(length,))
    source3_modify = source3[:, np.newaxis][length-1:]
    sums = np.sum(np.abs(windows - source3_modify),axis=1)
        
    mad = sums / length

    mad_series = pd.Series(index=range(len(source1)), dtype=float)
    mad_series[-len(mad):] = mad
    mad_series = mad_series.to_numpy()
    mcci = source2/mad_series/0.015
    return mcci

startTime = datetime.now()
# origin_data = dataframe.apply((lambda index: calculate_cci_origin(dataframe,index,dataframe['close'],dataframe['open'],3)),axis=1)
improved_data = pd.Series(calculate_cci_improved(dataframe['close'],dataframe['close'],dataframe['open'],3))
# test_data:pd.Series = origin_data == improved_data
endTime = datetime.now()
# print(improved_data[0]==origin_data[0])
print(f"speed is {round((endTime-startTime).microseconds * 0.001, 4)}ms")

speed is 1.445ms


In [5]:
def calculate_tr(index,high,low,close):
    high = pd.Series(high[:index+1])
    low = pd.Series(low[:index+1])
    close = pd.Series(close[:index+1])
    prev_close = close.shift(1).fillna(0)
    diff_h_n_l = high[index] - low[index]
    abs_value_h_n_c = abs(high[index] - prev_close[index])
    abs_value_l_n_c = abs(low[index] - prev_close[index])

    tr = max(max(diff_h_n_l,abs_value_h_n_c),abs_value_l_n_c)
    return tr

def calculate_directionalMovementPlus(index,high,low):
    high = pd.Series(high[:index+1])
    low = pd.Series(low[:index+1])
    prev_high = high.shift(1).fillna(0)
    prev_low = low.shift(1).fillna(0)

    diff_h_n_ph = high[index] - prev_high[index]
    diff_pl_n_l = prev_low[index] - low[index]
    dmp_value = max(diff_h_n_ph,0) if (diff_h_n_ph > diff_pl_n_l) else 0

    return dmp_value

def calculate_negMovement(index,high,low):
    high = pd.Series(high[:index+1])
    low = pd.Series(low[:index+1])
    prev_high = high.shift(1).fillna(0)
    prev_low = low.shift(1).fillna(0)

    diff_h_n_ph = high[index] - prev_high[index]
    diff_pl_n_l = prev_low[index] - low[index]
    negMovement = max(diff_pl_n_l,0) if (diff_pl_n_l > diff_h_n_ph) else 0
    return negMovement

def rescale(src, old_min, old_max, new_min, new_max):
    return new_min + (new_max - new_min) * (src - old_min) / np.maximum((old_max - old_min), 10e-10)

def n_adx_origin(highSrc, lowSrc, closeSrc,dataframe, n1):
    df = dataframe.copy()
    length = n1
    th = 20
    tr = df.apply((lambda x : calculate_tr(x.name,highSrc,lowSrc,closeSrc)),axis=1)
    directionalMovementPlus = df.apply((lambda x : calculate_directionalMovementPlus(x.name,highSrc,lowSrc)),axis=1)
    negMovement = df.apply((lambda x : calculate_negMovement(x.name,highSrc,lowSrc)),axis=1)

    trSmooth = np.zeros_like(closeSrc)
    trSmooth[0] = np.nan
    for i in range(0, len(tr)):
        trSmooth[i] = trSmooth[i-1] - trSmooth[i-1] / length + tr[i]

    smoothDirectionalMovementPlus = np.zeros_like(closeSrc)
    smoothDirectionalMovementPlus[0] = np.nan
    for i in range(0, len(directionalMovementPlus)):
        smoothDirectionalMovementPlus[i] = smoothDirectionalMovementPlus[i-1] - smoothDirectionalMovementPlus[i-1] / length + directionalMovementPlus[i]

    smoothnegMovement = np.zeros_like(closeSrc)
    smoothnegMovement[0] = np.nan
    for i in range(0, len(negMovement)):
        smoothnegMovement[i] = smoothnegMovement[i-1] - smoothnegMovement[i-1] / length + negMovement[i]
    
    diPositive = smoothDirectionalMovementPlus / trSmooth * 100
    diNegative = smoothnegMovement / trSmooth * 100
    dx = np.abs(diPositive - diNegative) / (diPositive + diNegative) * 100
    dx_series = pd.Series(dx)

    adx = dx_series.copy()
    adx.iloc[:length] = adx.rolling(length).mean().iloc[:length]
    adx = adx.ewm(alpha=(1.0/length),adjust=False).mean()
    return rescale(adx, 0, 100, 0, 1)

def calculate_tr_optimized(high,low,close):
    previos_close = np.roll(close,1)

    diff_h_n_l = high - low
    abs_value_h_n_c = np.abs(high - previos_close)
    abs_value_h_n_c[0] = abs(high[0] - 0)
    abs_value_l_n_c = np.abs(low - previos_close)
    abs_value_l_n_c[0] = abs(low[0] - 0)
    tr = np.maximum(np.maximum(diff_h_n_l,abs_value_h_n_c),abs_value_l_n_c)
    return tr

def calculate_directionalMovementPlus_optimized(high,low):
    prev_high = np.roll(high,1)
    prev_low = np.roll(low,1)

    diff_h_n_ph = high - prev_high
    diff_h_n_ph[0] = high[0] - 0
    diff_pl_n_l = prev_low - low
    diff_pl_n_l[0] = 0 - low[0]
    dmp_value = np.maximum(diff_h_n_ph, 0) * (diff_h_n_ph > diff_pl_n_l)
    return dmp_value

def calculate_negMovement_optimized(high,low):
    prev_high = np.roll(high,1)
    prev_low = np.roll(low,1)

    diff_h_n_ph = high - prev_high
    diff_h_n_ph[0] = high[0] - 0
    diff_pl_n_l = prev_low - low
    diff_pl_n_l[0] = 0 - low[0]
    negMovement = np.maximum(diff_pl_n_l, 0) * (diff_pl_n_l > diff_h_n_ph)
    return negMovement

def n_adx_optimized(highSrc:pd.Series, lowSrc:pd.Series, closeSrc:pd.Series, n1:int):
    length = n1
    highSrc_numpy = highSrc.to_numpy()
    lowSrc_numpy = lowSrc.to_numpy()
    closeSrc_numpy = closeSrc.to_numpy()
    
    tr = calculate_tr_optimized(highSrc_numpy,lowSrc_numpy,closeSrc_numpy)
    directionalMovementPlus = calculate_directionalMovementPlus_optimized(highSrc_numpy,lowSrc_numpy)
    negMovement = calculate_negMovement_optimized(highSrc_numpy,lowSrc_numpy)

    trSmooth = np.zeros_like(closeSrc_numpy)
    trSmooth[0] = np.nan
    for i in range(0, len(tr)):
        trSmooth[i] = trSmooth[i-1] - trSmooth[i-1] / length + tr[i]

    smoothDirectionalMovementPlus = np.zeros_like(closeSrc)
    smoothDirectionalMovementPlus[0] = np.nan
    for i in range(0, len(directionalMovementPlus)):
        smoothDirectionalMovementPlus[i] = smoothDirectionalMovementPlus[i-1] - smoothDirectionalMovementPlus[i-1] / length + directionalMovementPlus[i]

    smoothnegMovement = np.zeros_like(closeSrc)
    smoothnegMovement[0] = np.nan
    for i in range(0, len(negMovement)):
        smoothnegMovement[i] = smoothnegMovement[i-1] - smoothnegMovement[i-1] / length + negMovement[i]

    diPositive = smoothDirectionalMovementPlus / trSmooth * 100
    diNegative = smoothnegMovement / trSmooth * 100
    dx = np.abs(diPositive - diNegative) / (diPositive + diNegative) * 100
    dx_series = pd.Series(dx)

    adx = dx_series.copy()
    adx.iloc[:length] = adx.rolling(length).mean().iloc[:length]
    adx = adx.ewm(alpha=(1.0/length),adjust=False).mean()
    return rescale(adx, 0, 100, 0, 1)

startTime = datetime.now()
# origin_data = n_adx_origin(dataframe['high'],dataframe['low'],dataframe['close'],dataframe,3)
improved_data = pd.Series(n_adx_optimized(dataframe['high'],dataframe['low'],dataframe['close'],3))
# test_data:pd.Series = origin_data == improved_data
endTime = datetime.now()
print(f"speed is {round((endTime-startTime).microseconds * 0.001, 4)}ms")

speed is 8.833ms


In [6]:
def compare_value_origin(index, dataframe, dataframe_shifted,future_count):
    df_data = dataframe['close']
    df_shifted_data = dataframe_shifted['close']
    if(index >= len(df_data)-future_count):
        return np.nan
    if (df_shifted_data[index] >= df_data[index]):
        return 1
    else:
        return -1
    
def compare_value_improved(dataframe:pd.DataFrame, dataframe_shifted:pd.DataFrame,future_count:int):
    df_data = dataframe['close'].to_numpy()
    df_shifted_data = dataframe_shifted['close'].to_numpy()

    result = np.where(df_shifted_data >= df_data , 1 , -1)
    result = result[:-future_count]
    result_series = pd.Series(index=range(len(df_data)), dtype=float)
    result_series[:len(result)] = result

    return result_series


startTime = datetime.now()
future_count = 1
df_shifted = dataframe.shift(future_count)
dataframe['y_train'] = (df_shifted['close'] < dataframe['close']).astype(int) * 2 - 1
# dataframe['y_train_origin'] = dataframe.apply(lambda x: compare_value_origin(x.name,dataframe,df_shifted,future_count),axis=1)
# dataframe['y_train_improved'] = compare_value_improved(dataframe,df_shifted,future_count)
endTime = datetime.now()
print(dataframe)
print(f"speed is {round((endTime-startTime).microseconds * 0.001, 4)}ms")

                   date      open      high       low     close      volume  \
0   2024-04-14 01:50:00  0.014971  0.015218  0.014958  0.015209  11834259.0   
1   2024-04-14 01:55:00  0.015209  0.015265  0.015079  0.015095  13997433.0   
2   2024-04-14 02:00:00  0.015094  0.015244  0.015087  0.015147  17827585.0   
3   2024-04-14 02:05:00  0.015144  0.015207  0.015045  0.015046  16165649.0   
4   2024-04-14 02:10:00  0.015048  0.015061  0.014945  0.014983  12208130.0   
..                  ...       ...       ...       ...       ...         ...   
994 2024-04-17 12:40:00  0.013956  0.013967  0.013887  0.013956  19395595.0   
995 2024-04-17 12:45:00  0.013955  0.014025  0.013909  0.014004  14973104.0   
996 2024-04-17 12:50:00  0.014004  0.014086  0.014004  0.014042  14955563.0   
997 2024-04-17 12:55:00  0.014043  0.014070  0.014020  0.014054   8911663.0   
998 2024-04-17 13:00:00  0.014054  0.014148  0.014049  0.014127  12882417.0   

     y_train  
0         -1  
1         -1  
2     

In [7]:
startTime = datetime.now()
improved_data = pd.Series(normalize_optimized(0,1,dataframe['high']))
improved_data = pd.Series(calculate_cci_improved(dataframe['close'],dataframe['close'],dataframe['open'],3))
improved_data = pd.Series(n_adx_optimized(dataframe['high'],dataframe['low'],dataframe['close'],3))
endTime = datetime.now()
print(f"speed is {round((endTime-startTime).microseconds * 0.001, 4)}ms")

speed is 15.122ms
